In [12]:
import mlrose_hiive as mh
from mlrose_hiive.runners import SARunner, GARunner, MIMICRunner, RHCRunner, NNGSRunner
from mlrose_hiive import QueensGenerator, QueensOpt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [13]:
def model_evaluation(y_test, y_pred):
   accuracy = metrics.accuracy_score(y_test, y_pred)
   f1 = metrics.f1_score(y_test, y_pred)
   return accuracy, f1

In [14]:
cars = pd.read_csv('data/car.data', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'target'])
cars['target'] = cars['target'].apply(lambda x: 0 if x == 'unacc' else 1)
Y = cars['target']

skencoder = OneHotEncoder(drop='first',sparse_output=False)
skencoder.set_output(transform="pandas")
ohe_cars = skencoder.fit_transform(cars.loc[:,['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']])
ohe_cars['target'] = Y
cars_x = ohe_cars.loc[:, ohe_cars.columns != 'target']
cars_y = ohe_cars['target']

X_train, X_test, y_train, y_test = train_test_split(cars_x, cars_y, test_size = 0.2, random_state=0)



In [15]:
params_rhc ={
    'max_iters': [500],
    'activation': [mh.relu],
    'restarts': [10,20,30],
}

nn_rhc = NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='nn_test_rhc',
    algorithm=mh.algorithms.rhc.random_hill_climb,
    grid_search_parameters=params_rhc,
    iteration_list=2 ** np.arange(11),
    hidden_layer_sizes=[[180]],
    early_stopping=False,
    max_attempts=100,
    seed=0,
    n_jobs=5,
    output_directory=None
)

run_stats_rhc, curves_rhc, cv_results_rhc, grid_search_cv_rhc = nn_rhc.run()

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [24]:
cv_results_rhc

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_max_iters,param_restarts,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,46.331774,0.162126,0.0018,0.00040,relu,[180],500,10,{'activation': <function relu at 0x000001172CC...,0.493541,...,0.553984,0.042279,2,0.570193,0.545135,0.560848,0.551413,0.539496,0.553417,0.010982
1,106.111942,0.209169,0.0014,0.00049,relu,[180],500,20,{'activation': <function relu at 0x000001172CC...,0.491802,...,0.558059,0.037689,1,0.572028,0.552383,0.562102,0.554679,0.551558,0.558550,0.007697
2,185.360166,0.197668,0.0014,0.00049,relu,[180],500,30,{'activation': <function relu at 0x000001172CC...,0.494411,...,0.550593,0.035575,3,0.562510,0.549608,0.558376,0.548146,0.541109,0.551950,0.007619


In [18]:
curves_rhc

,Iteration,Time,Fitness,FEvals,max_iters,restarts,init_state,algorithm,activation,bias,early_stopping,clip_max,hidden_layer_sizes,learning_rate_init,max_attempts,current_restart
0,0,0.009810,2.729372,1.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,0
1,1,0.022320,2.729372,2.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,0
2,2,0.033986,2.729372,3.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,0
3,3,0.043952,2.729372,4.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,0
4,4,0.053420,2.727414,6.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10496,496,3.260063,1.660393,15734.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,20
10497,497,3.268233,1.659977,15736.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,20
10498,498,3.272947,1.659087,15738.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,20
10499,499,3.282385,1.659087,15739.0,500,20,"[0.0976270078546495, 0.43037873274483895, 0.20...",rhc,relu,True,False,1.000000e+10,[180],0.1,500,20


In [23]:
y_pred_rhc = grid_search_cv_rhc.predict(X_test)
acc_rhc, f1_rhc = model_evaluation(y_test, y_pred_rhc)
print('Accuracy: ', acc_rhc)
print('F1: ', f1_rhc)

Accuracy:  0.569364161849711
F1:  0.46209386281588444


In [25]:
params_sa = {
    'max_iters': [500],
    'activation': [mh.relu],
    'temperatures': [2.0, 1.5, 1.0, 0.75, 0.66, 0.5, 0.1]
}

nn_sa = NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='nn_test_sa',
    algorithm=mh.algorithms.sa.simulated_annealing,
    grid_search_parameters=params_sa,
    iteration_list=2 ** np.arange(11),
    hidden_layer_sizes=[[180]],
    early_stopping=False,
    max_attempts=100,
    seed=0,
    n_jobs=5,
    output_directory=None
)

run_stats_sa, curves_sa, cv_results_sa, grid_search_cv_sa = nn_sa.run()


Fitting 5 folds for each of 7 candidates, totalling 35 fits


In [26]:
y_pred_sa = grid_search_cv_sa.predict(X_test)
acc_sa, f1_sa = model_evaluation(y_test, y_pred_sa)
print('Accuracy: ', acc_sa)
print('F1: ', f1_sa)

Accuracy:  0.4682080924855491
F1:  0.48022598870056493


In [4]:
params_ga = {
    'max_iters': [500],
    'activation': [mh.relu],
    'population_sizes': [150],
    'mutation_rates': [0.05]
}

nn_ga = NNGSRunner(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    experiment_name='nn_test_ga',
    algorithm=mh.algorithms.ga.genetic_alg,
    grid_search_parameters=params_ga,
    iteration_list=2 ** np.arange(11),
    hidden_layer_sizes=[[180]],
    early_stopping=False,
    max_attempts=100,
    seed=0,
    n_jobs=5,
    output_directory=None
)

run_stats_ga, curves_ga, cv_results_ga, grid_search_cv_ga = nn_ga.run()

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [5]:
y_pred_ga = grid_search_cv_ga.predict(X_test)
acc_ga, f1_ga = model_evaluation(y_test, y_pred_ga)
print('Accuracy: ', acc_ga)
print('F1: ', f1_ga)

Accuracy:  0.8728323699421965
F1:  0.8
